In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px
%matplotlib notebook 


In [ ]:
tweet_analysis=pd.read_csv("tweetsnew.csv")
df=tweet_analysis.copy()
df.head()  

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.nunique()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df2=df.drop(["Unnamed: 0","source of tweet", "Date","User","Tweet","language","media","mentioned user in tweet"],axis=1)

In [ ]:
df2.max()

In [ ]:
corr = df2.corr()
corr.style.background_gradient(cmap ='coolwarm')

# datetime which is in the ISO 8601 format to date format
yyyy-mm-dd HH:MM:SS and the time zone offset in the format "+/-HH:MM) to "yyyy-mm-dd".

In [ ]:
df['Date'] = pd.to_datetime(df['Date']).dt.date
df

In [ ]:
date=pd.DataFrame(df['Date'].value_counts())
date.plot.bar()

In [ ]:
language=pd.DataFrame(df['language'].value_counts())
language

In [ ]:
Top_5=df['language'].value_counts().head(5)
fig, ax = plt.subplots()
ax.pie(Top_5.values, labels=Top_5.index)

# Set the title of the chart
plt.title('Top_5 languages')

# Set the aspect ratio to be equal so the pie chart is circular
ax.axis('equal')

# Remove the labels that overlap
plt.legend(Top_5.index, loc="best")

# Show the chart
plt.show()

In [15]:
df["media"]=df["media"].str.split("[").str[1]

In [ ]:
df["media_shared"]=df["media"].str.split("(").str[0]
df

In [ ]:
media_counts=pd.DataFrame(df['media_shared'].value_counts())
media_counts


In [ ]:
def plot_bar(x,y,title,x_label,y_label):
    fig = go.Figure(data=[go.Bar(
                x=x,
                y=y,
                text=y,
                textposition='auto',
            )])

    fig.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title=y_label
    )

    fig.show()


In [ ]:
import plotly.graph_objects as go
plot_bar(df['User'].value_counts().index[0:20],
         df['User'].value_counts().values[0:20],
         "Top 20 Users by the number of tweets",
         "User",
         "Tweet Count")

In [ ]:
plot_bar(df['source of tweet'].value_counts().index[0:20],
         df['source of tweet'].value_counts().values[0:20],
         "Top 20 Tweeter Sources by the number of tweets",
         "Source",
         "Tweet Count")


In [ ]:
df3=df.drop([ "source of tweet","media","mentioned user in tweet"],axis=1)
df3= df3.loc[df3['language']=="en"] 
df3

# fine grained sentiment analysis 

# Beginner technique-VADER


In [22]:
import nltk
from nrclex import NRCLex
import vaderSentiment
# calling SentimentIntensityAnalyzer object
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [23]:
def sentiment_analyzer_score(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [ ]:
df3['Vader_scores'] = df3['Tweet'].apply(lambda review: analyser.polarity_scores(review))
df3.head()


In [ ]:
df3['vader_compound']  = df3['Vader_scores'].apply(lambda score_dict: score_dict['compound'])
df3['vader_negative']  = df3['Vader_scores'].apply(lambda score_dict: score_dict['neg'])
df3['vader_neutral']  = df3['Vader_scores'].apply(lambda score_dict: score_dict['neu'])
df3['vader_positive']  = df3['Vader_scores'].apply(lambda score_dict: score_dict['pos'])
df3.head()

In [26]:
def sentiment_analysis2(score):
    if score <=-0.8:
        return 'extreme negative sentiment'
    elif score > -0.8 and score < 0:
        return 'weak negative sentiment'
    elif score == 0:
        return 'Neutral'
    elif score >0 and score <0.8 :
        return 'Weak positive sentiment'
    else: 
        return 'extreme positive sentiment'

In [ ]:
df3['Vader_scores'] = df3['vader_compound'].apply(sentiment_analysis2)
df3.head()

In [ ]:
vadersentiment_count = df3['Vader_scores'].value_counts()
vadersentiment_count


# Beginner technique-TEXTBLOB:



In [ ]:
from textblob import TextBlob
import unicodedata as uni
import re
def CleanTxt(text):
    text = re.sub(r'@[\w]+','',text)
    text = re.sub(r'#','',text)
    text = re.sub(r'RT[\s]+','',text) # \s for space and + for one or more
    text = re.sub(r'https?:\/\/\S+', '', text)
    return text

df3["Tweet_textblob"]=df3["Tweet"].apply(CleanTxt)
df3.tail()

In [ ]:
def CleanTett(text):
    text = uni.normalize('NFKD', text)
    text = text.lower() 
    return text

df3["Tweet_textblob"]=df3["Tweet_textblob"].apply(CleanTett)
df3.tail()

In [ ]:
from cleantext import clean
def emoji_to_text(text):
    if type(text) != float:
        return clean(text, no_emoji=True)
    else:
        return text
    
df3["Tweet_textblob"]=df3["Tweet_textblob"].apply(emoji_to_text)
df3

In [32]:
def getSubjectivity(text):
    # will get the subjectivity of the tweet
    blob = TextBlob(text)
    return blob.sentiment.subjectivity

In [33]:
def getPolarity(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

In [ ]:
df3['subjectivity'] = df3['Tweet_textblob'].apply(getSubjectivity)
df3['polarity'] = df3['Tweet_textblob'].apply(getPolarity)
df3

In [35]:
def sentiment_analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [ ]:
df3['Text_blobsentiment'] = df3['polarity'].apply(sentiment_analysis)
df3.tail()

In [ ]:
sentiment_count = df3['Text_blobsentiment'].value_counts()
sentiment_count

# Advanced technique -transformers(DISTILBERT)


In [52]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
import torch
from scipy.special import softmax
import csv
import urllib.request

In [53]:
from transformers import pipeline
def FunctiondistBERTSentiment(inpText):
    return(SentimentClassifier(inpText)[0]['label'])



SentimentClassifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
df3['DistilBERT_Sentiment']=df3['Tweet'].apply(FunctiondistBERTSentiment)
df3.head()

In [ ]:
plt.figure(figsize=(6,6))
sns.countplot(x='DistilBERT_Sentiment',data=df3)

# WORDCLOUD

In [56]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
pd.options.mode.chained_assignment = None

In [ ]:
df_positive = ' '.join([Text for Text in df3[df3['DistilBERT_Sentiment']=='POSITIVE']['Tweet_word2vec']])
wordcloud = WordCloud(width=800, height=400, background_color=
                       "white").generate(df_positive)
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = "yellow") 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
df_negative = ' '.join([Text for Text in df3[df3['DistilBERT_Sentiment']=='NEGATIVE']['Tweet_word2vec']])
wordcloud = WordCloud(width=800, height=400, background_color=
                       "white").generate(df_negative)
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = "yellow") 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

# EMOTION DETECTION 
USING NRC LEXICON


In [ ]:
def emotion_detection(sentence):
    tweett = NRCLex(sentence)
    return tweett.affect_dict

In [ ]:
df3['affected_dictionary'] = df3['Tweet'].apply(emotion_detection)
df3

In [ ]:
def top_emotionse(sentence):

        return NRCLex(sentence).top_emotions
df3['top_emotions_score'] = df3['Tweet'].apply(top_emotionse)
df3

In [ ]:
def top_emotionss(sentence):
    tweett = NRCLex(sentence)
    if tweett.top_emotions[0][1] == 0.0:
        return "No emotion"
    else:
        return tweett.top_emotions[0][0]
df3['top_emotions'] = df3['Tweet'].apply(top_emotionss)
df3


In [ ]:
plt.figure(figsize=(6,6))
sns.countplot(x='top_emotions',data=df3)
plt.xticks(rotation=70)

# emotions categorised under bert (positive & negative)


In [ ]:
df002 = df3.loc[(df3['DistilBERT_Sentiment']=='NEGATIVE')]
emotion_Distlbert_negative  = df002['top_emotions'].value_counts()
emotion_Distlbert_negative 

In [ ]:
df001 = df3.loc[(df3['DistilBERT_Sentiment']=='POSITIVE')]
emotion_Distlbert_positive = df001['top_emotions'].value_counts()
emotion_Distlbert_positive 

# ASPECT BASED SENTIMENT ANALYSIS
zero shot classification for identifying potential vulnerable people at the time of crisis


In [75]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [76]:
health_labels = ["sadness",'crisis','disorder','pain','optimistic','personal','non-personal']

In [ ]:
df6 = df3.loc[(df3['top_emotions']=="sadness")&(df3['DistilBERT_Sentiment']=='NEGATIVE')]
df6

In [78]:
def ZERO_SHOT(inpText):
    return(classifier( inpText,health_labels))

In [ ]:
df6['health_matters'] = df6['Tweet'].apply(ZERO_SHOT)
df6    


In [80]:
df6.to_excel("textfiledf70.xlsx",sheet_name="new_sample")